1. Réalisez un automate élémentaire.

Il faut d'abord fixer quelques règles, comme par exemple pour le cas ou une cellule atteint un bord mais qu'il reste encore des lignes a générer.
Je vais partir sur un principe de bords ouverts, donc si une cellule arrive a un bord (exemple: bord gauche), alors sont voisin de gauche sera la cellule a l'extreme droite et vice versa.
Pour éviter de se trainer une liste de liste de x*y case, je vais juste créer une liste vide qui seras la grille des générations, et à chaque génération, je vais créer une liste pour la generation actuel et l'ajouter à la liste des générations.
On aura donc 3 paramètres: la regle, la génération initial et le nombre de génération voulu.

In [1]:
def automate_elementaire(regle, initial, nb_gen):
    generations=[]
    regle_bin = [int(bit) for bit in format(regle, '08b')[::-1]]
    size = len(initial)
    generations.append(initial)
    for i in range(nb_gen):
        generation = []
        for j in range(size):
            motif = int(str(generations[i][(j-1)%size]) + str(generations[i][j]) + str(generations[i][(j+1)%size]),2)
            generation.append(regle_bin[motif])
        generations.append(generation)
    return generations

2. Vous pouvez prendre la règle d'évolution de votre choix, et faire marcher cet automate, en mode texte, dans un terminal.

In [2]:
automate = automate_elementaire(30, [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0], 10)

for gen in automate:
    pimp_gen = ['🟥' if bit == 0 else '🟩' for bit in gen]
    print(pimp_gen)
    print()

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟥', '🟩', '🟩', '🟩', '🟩', '🟩', '🟩', '🟥', '🟥']

['🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟥']

['🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟩', '🟩', '🟩']

['🟥', '🟥', '🟥', '🟩', '🟥', '🟥',

3. Pour l'améliorer, on peut penser demander à l'utilisateur son choix de règle, puis en faire une représentation graphique.

Pour l'interface graphique je vais utiliser pygame.

In [4]:
import pygame
import sys
import math

# Initialisation de Pygame
pygame.init()

def lancer_automate():
    pass

# Fonction pour afficher la fenêtre Pygame avec la règle et les boutons
def afficher_fenetre(regle):
    width, height = 800, 800
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Programme Pygame")

    # Couleurs
    white = (255, 255, 255)
    black = (0, 0, 0)
    

    # Police
    font = pygame.font.Font(None, 20)

    # Grille
    nb_lignes, nb_colonnes = 1, 30
    cell_size = width / nb_colonnes
    grille = [[0] * nb_colonnes for _ in range(nb_lignes)]

    # Boucle principale de la fenêtre Pygame
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                x,y = event.pos
                
                print (x,y)
                if cell_size < y < 2*cell_size:
                    ligne = 0
                    colonne = math.floor(x/cell_size)
                    grille[ligne][colonne] = (grille[ligne][colonne] + 1 ) % 2
                else:
                    print("not grid")

            # Dessiner le fond
            screen.fill(white)

        # Dessiner la règle
        text_regle = font.render(f"Règle : {regle}", True, (0, 0, 0))
        screen.blit(text_regle, (350, 10))
        
        # Dessiner la grille

        # Dessiner les cellules
        for ligne in range(nb_lignes):
            for colonne in range(nb_colonnes):
                couleur = black if grille[ligne][colonne] == 1 else white
                pygame.draw.rect(screen, couleur, (colonne * cell_size, (ligne + 1) * cell_size, cell_size, cell_size))

        # Mettre à jour l'affichage
        pygame.display.flip()

        # Limiter la vitesse de la boucle
        pygame.time.Clock().tick(60)

# Boucle principale du programme
while True:
    numero_regle = input("Entrez le numéro de règle (ou 'quitter' pour quitter) : ")
    
    if numero_regle.lower() == 'quitter':
        sys.exit()

    try:
        numero_regle = int(numero_regle)
        # Afficher la fenêtre Pygame avec la règle et les boutons
        redemander_regle = afficher_fenetre(numero_regle)
        if not redemander_regle:
            continue  # Redemander le numéro de règle
    except ValueError:
        print("Veuillez entrer un numéro de règle valide.")


356 61
317 66
265 47
not grid
260 70
307 72
329 69
392 67
418 67
445 65
471 64
391 64
391 64
463 65
463 65
490 66
518 66
546 66
569 66
597 66
622 68
657 69
681 69
706 67
739 65
767 62
787 63
304 59
276 59
280 36
not grid
266 7
253 12
230 12
200 13
166 16
125 16
55 14
55 5
57 33
not grid
57 33
not grid
57 33
not grid
57 34
not grid
57 34
not grid
57 34
not grid
92 200
119 288
186 329


SystemExit: 

C:\Users\lferry\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
